<a href="https://colab.research.google.com/github/trakshala457/DDR_Report_Generator/blob/main/DDR_Report_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pypdf pdfplumber reportlab google-generativeai
print("✅ All dependencies installed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 70.4 MB/s eta 0:00:00
✅ All dependencies installed!


In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
print("API key done")

API key done


In [ ]:
from google.colab import files

print("📁 Please upload your TWO PDF files:")
print("   1) Inspection Report PDF")
print("   2) Thermal Images PDF")
print()

uploaded = files.upload()
uploaded_names = list(uploaded.keys())

print(f"\n✅ Uploaded files: {uploaded_names}")

if len(uploaded_names) < 2:
    print("⚠️  Warning: Expected 2 files. You can re-run this cell to upload more.")

📁 Please upload your TWO PDF files:
   1) Inspection Report PDF
   2) Thermal Images PDF



Saving Thermal Images.pdf to Thermal Images.pdf
Saving Insepection Report.pdf to Insepection Report.pdf

✅ Uploaded files: ['Thermal Images.pdf', 'Insepection Report.pdf']


In [ ]:
# ── Manually assign files (edit filenames to match your uploads) ────────────
# The script will try to auto-detect, but you can override below.

INSPECTION_PDF = None  # e.g. "Insepection_Report.pdf"
THERMAL_PDF    = None  # e.g. "Thermal_Images.pdf"

# ── Auto-detect if not manually set ────────────────────────────────────────
if not INSPECTION_PDF or not THERMAL_PDF:
    for name in uploaded_names:
        n = name.lower()
        if any(kw in n for kw in ["inspect", "report", "form"]):
            INSPECTION_PDF = name
        elif any(kw in n for kw in ["thermal", "therm", "image"]):
            THERMAL_PDF = name

    # Fallback: assign by order
    if not INSPECTION_PDF and not THERMAL_PDF and len(uploaded_names) == 2:
        INSPECTION_PDF, THERMAL_PDF = uploaded_names[0], uploaded_names[1]
    elif not INSPECTION_PDF:
        INSPECTION_PDF = [n for n in uploaded_names if n != THERMAL_PDF][0]
    elif not THERMAL_PDF:
        THERMAL_PDF = [n for n in uploaded_names if n != INSPECTION_PDF][0]

print(f"Inspection Report : {INSPECTION_PDF}")
print(f"Thermal Report    : {THERMAL_PDF}")

Inspection Report : Insepection Report.pdf
Thermal Report    : Thermal Images.pdf


In [ ]:
import pdfplumber

def extract_text_from_pdf(filepath: str) -> str:
    """
    Extract all text from a PDF using pdfplumber.
    Falls back to pypdf if pdfplumber yields nothing.
    Works for both text-based and scanned-with-text-layer PDFs.
    """
    text_pages = []
    try:
        with pdfplumber.open(filepath) as pdf:
            for i, page in enumerate(pdf.pages):
                page_text = page.extract_text() or ""
                # Also try to grab any table data
                tables = page.extract_tables()
                table_text = ""
                for table in tables:
                    for row in table:
                        if row:
                            cleaned = [str(cell).strip() if cell else "" for cell in row]
                            table_text += " | ".join(cleaned) + "\n"
                combined = page_text
                if table_text and table_text not in page_text:
                    combined += "\n" + table_text
                text_pages.append(f"--- Page {i+1} ---\n{combined}")
    except Exception as e:
        print(f"  pdfplumber error: {e} — trying pypdf fallback")
        from pypdf import PdfReader
        reader = PdfReader(filepath)
        for i, page in enumerate(reader.pages):
            text_pages.append(f"--- Page {i+1} ---\n{page.extract_text() or ''}")

    return "\n\n".join(text_pages)


print("Extracting text from Inspection Report...")
inspection_text = extract_text_from_pdf(INSPECTION_PDF)
print(f"   Extracted {len(inspection_text):,} characters from {len(inspection_text.split(chr(10)))} lines")

print("\nExtracting text from Thermal Report...")
thermal_text = extract_text_from_pdf(THERMAL_PDF)
print(f" Extracted {len(thermal_text):,} characters from {len(thermal_text.split(chr(10)))} lines")

Extracting text from Inspection Report...
   Extracted 8,450 characters from 364 lines

Extracting text from Thermal Report...
 Extracted 529 characters from 89 lines


In [ ]:
# Quick sanity check — shows first 1000 chars of each
print("=" * 60)
print("INSPECTION REPORT (first 1000 chars):")
print("=" * 60)
print(inspection_text[:1000])

print("\n" + "=" * 60)
print("THERMAL REPORT (first 1000 chars):")
print("=" * 60)
print(thermal_text[:1000])

INSPECTION REPORT (first 1000 chars):
--- Page 1 ---
Inspection Form
Complete
Score 85.71% Flagged items 1 Actions 0
Customer Name
Mobile:
Email:
Address:
Property Age (In years):
Property Type: Flat
Floors: 11
Previous Structural audit done No
Previous Repair work done No
Inspection Date and Time: 27.09.2022 14:28 IST
Inspected By: Krushna & Mahesh
Score | 85.71% | Flagged items | 1 | Actions | 0
 |  |  |  |  | 

Flat


No


No



--- Page 2 ---
Flagged items 1 flagged
Checklists / Inspection Checklists
Checklist: WC
External wall

--- Page 3 ---
Site Details
Hall, Bedroom, Kitchen, Master
Impacted Areas/Rooms Bedroom, Parking Area,
Common bathroom
Impacted Area
Impacted Area 1
Negative side Description Hall Skirting level Dampness
Negative side photographs
Photo 1 Photo 2 Photo 3 Photo 4 Photo 5 Photo 6
Photo 7
Common Bathroom tile
Positive side Description
hollowness
Positive side photographs
Photo 8 Photo 9 Photo 10 Photo 11
Impacted Area 2
Bedroom Skirting level
Negative side Desc

In [ ]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

# ── Prompt Engineering ──────────────────────────────────────────────────────
SYSTEM_PROMPT = """
You are a professional building diagnostics expert. Your job is to read raw inspection
and thermal report data and produce a structured, client-friendly Detailed Diagnostic
Report (DDR).

STRICT RULES:
- Do NOT invent or assume any facts not present in the documents.
- If information is missing → write exactly: Not Available
- If information conflicts between the two reports → explicitly mention the conflict.
- Do NOT duplicate points that appear in both reports — merge them logically.
- Use simple, plain English. Avoid excessive technical jargon.
- Be specific: mention room names, flat numbers, and temperature values where available.
"""

DDR_PROMPT = f"""
{SYSTEM_PROMPT}

=== INSPECTION REPORT DATA ===
{inspection_text[:15000]}

=== THERMAL REPORT DATA ===
{thermal_text[:8000]}

=== YOUR TASK ===
Using the above data, generate a complete DDR (Detailed Diagnostic Report) with
EXACTLY this structure. Use the headers exactly as written:

---
DETAILED DIAGNOSTIC REPORT (DDR)
---

SECTION 1: PROPERTY ISSUE SUMMARY
[Brief overall summary of findings — 3 to 5 sentences. Include inspection date,
property type, flat number if available, and general severity.]

SECTION 2: AREA-WISE OBSERVATIONS
[For each affected area, write a paragraph covering:
 - What was observed visually (from inspection report)
 - Thermal readings if available for that area (hotspot/coldspot temperatures)
 - Whether leakage is constant or intermittent]

SECTION 3: PROBABLE ROOT CAUSE
[Explain in plain language what is most likely causing each issue.
 Connect the visual observations to the thermal data where possible.]

SECTION 4: SEVERITY ASSESSMENT
[Rate each issue as: Low / Moderate / High / Critical
 Explain your reasoning for each rating.]

SECTION 5: RECOMMENDED ACTIONS
[List specific repair actions for each area. Be actionable — mention what type
 of repair is needed (e.g. re-grouting, waterproofing, plumbing fix).]

SECTION 6: ADDITIONAL NOTES
[Any patterns noticed, preventive advice, or cross-area observations.]

SECTION 7: MISSING OR UNCLEAR INFORMATION
[List any data that was absent or unclear. Use 'Not Available' for each missing item.]
"""

print("Sending to Gemini AI for analysis...")
model = genai.GenerativeModel("gemini-2.5-flash")
response = model.generate_content(DDR_PROMPT)
ddr_text = response.text

print("\n DDR Report generated successfully!")
print(f"   Total characters: {len(ddr_text):,}")

Sending to Gemini AI for analysis...

 DDR Report generated successfully!
   Total characters: 12,282


In [ ]:
print(ddr_text)

---
DETAILED DIAGNOSTIC REPORT (DDR)
---

SECTION 1: PROPERTY ISSUE SUMMARY
This report details the diagnostic findings for Flat No. 103, a unit within an 11-floor apartment building. The inspection was conducted on 27.09.2022 by Krushna & Mahesh. The findings indicate widespread dampness and leakage issues, primarily originating from compromised waterproofing and plumbing in the unit's bathrooms, external wall defects, and potential leakage from the adjacent flat above. The overall severity of the observed issues ranges from moderate to high, requiring prompt and targeted repairs to prevent further damage.

SECTION 2: AREA-WISE OBSERVATIONS

**Hall (Flat No. 103)**
Dampness was observed at the skirting level in the Hall. The probable source is identified as the Common Bathroom of Flat No. 103, where gaps between tile joints and tile hollowness were noted. Leakage due to concealed plumbing and damage to the Nahani trap/brickbat coba in the Common Bathroom is reported as occurring "All 

In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, HRFlowable, Table, TableStyle
)
from reportlab.lib.enums import TA_CENTER, TA_LEFT, TA_JUSTIFY
from datetime import datetime


def save_ddr_as_pdf(ddr_text: str, output_path: str = "DDR_Report.pdf"):
    """Convert the DDR text to a clean, professional PDF."""

    doc = SimpleDocTemplate(
        output_path,
        pagesize=A4,
        rightMargin=2*cm, leftMargin=2*cm,
        topMargin=2*cm, bottomMargin=2*cm
    )

    styles = getSampleStyleSheet()

    # Custom styles
    title_style = ParagraphStyle(
        'DDRTitle',
        parent=styles['Title'],
        fontSize=20,
        textColor=colors.HexColor('#1a3c5e'),
        spaceAfter=6,
        alignment=TA_CENTER
    )
    subtitle_style = ParagraphStyle(
        'Subtitle',
        parent=styles['Normal'],
        fontSize=10,
        textColor=colors.HexColor('#555555'),
        alignment=TA_CENTER,
        spaceAfter=4
    )
    section_style = ParagraphStyle(
        'SectionHeader',
        parent=styles['Heading1'],
        fontSize=13,
        textColor=colors.white,
        backColor=colors.HexColor('#1a3c5e'),
        spaceBefore=14,
        spaceAfter=6,
        leftIndent=-10,
        rightIndent=-10,
        leading=20
    )
    body_style = ParagraphStyle(
        'DDRBody',
        parent=styles['Normal'],
        fontSize=10,
        leading=15,
        spaceAfter=6,
        alignment=TA_JUSTIFY
    )
    bullet_style = ParagraphStyle(
        'DDRBullet',
        parent=styles['Normal'],
        fontSize=10,
        leading=14,
        leftIndent=16,
        spaceAfter=3
    )

    story = []

    # ── Header ────────────────────────────────────────────────────────────
    story.append(Spacer(1, 0.3*cm))
    story.append(Paragraph("DETAILED DIAGNOSTIC REPORT", title_style))
    story.append(Paragraph("Property Inspection & Thermal Analysis", subtitle_style))
    story.append(Paragraph(
        f"Generated on: {datetime.now().strftime('%d %B %Y, %H:%M')}  |  "
        f"System: AI-Assisted DDR Generator",
        subtitle_style
    ))
    story.append(HRFlowable(width="100%", thickness=2,
                             color=colors.HexColor('#1a3c5e'), spaceAfter=10))
    story.append(Spacer(1, 0.4*cm))

    # ── Parse and render DDR sections ─────────────────────────────────────
    lines = ddr_text.split('\n')
    in_section = False

    for line in lines:
        stripped = line.strip()
        if not stripped:
            story.append(Spacer(1, 0.2*cm))
            continue

        # Skip raw document separators
        if stripped in ['---', '===']:
            continue

        # Section headers (SECTION N:)
        if stripped.startswith('SECTION') and ':' in stripped:
            in_section = True
            # Format: "SECTION 1: PROPERTY ISSUE SUMMARY"
            header_text = stripped
            story.append(Spacer(1, 0.3*cm))
            story.append(Paragraph(f"  {header_text}  ", section_style))
            story.append(Spacer(1, 0.2*cm))

        # "DETAILED DIAGNOSTIC REPORT (DDR)" title line inside text
        elif 'DETAILED DIAGNOSTIC REPORT' in stripped.upper() and stripped.isupper():
            pass  # Already rendered in header

        # Bullet-like lines starting with - or *
        elif stripped.startswith(('-', '*', '•')):
            clean = stripped.lstrip('-*• ').strip()
            # Escape special XML chars
            clean = clean.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
            story.append(Paragraph(f"• {clean}", bullet_style))

        # Sub-headers (lines ending with : that are short)
        elif stripped.endswith(':') and len(stripped) < 60 and stripped == stripped.upper():
            clean = stripped.replace('&', '&amp;')
            story.append(Paragraph(
                f"<b>{clean}</b>",
                ParagraphStyle(
                    'SubHeader',
                    parent=styles['Normal'],
                    fontSize=11,
                    textColor=colors.HexColor('#1a3c5e'),
                    spaceBefore=8, spaceAfter=3
                )
            ))

        # Bold key-value pairs (e.g. "Area: Hall")
        elif ':' in stripped and len(stripped) < 80 and not stripped.startswith('http'):
            parts = stripped.split(':', 1)
            key = parts[0].strip().replace('&', '&amp;').replace('<', '&lt;')
            val = parts[1].strip().replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
            if val:  # only render as key-value if there's a value
                story.append(Paragraph(f"<b>{key}:</b> {val}", body_style))
            else:
                story.append(Paragraph(f"<b>{stripped.replace('&', '&amp;')}</b>", body_style))

        # Regular body paragraph
        else:
            safe = stripped.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
            story.append(Paragraph(safe, body_style))

    # ── Footer note ───────────────────────────────────────────────────────
    story.append(Spacer(1, 0.6*cm))
    story.append(HRFlowable(width="100%", thickness=1,
                             color=colors.HexColor('#cccccc')))
    story.append(Paragraph(
        "This report was generated by an AI-assisted DDR system. "
        "All findings are based solely on the provided inspection and thermal documents. "
        "Please consult a qualified structural engineer before taking remedial action.",
        ParagraphStyle('Footer', parent=styles['Normal'],
                       fontSize=8, textColor=colors.grey,
                       alignment=TA_CENTER, spaceBefore=6)
    ))

    doc.build(story)
    print(f"✅ DDR PDF saved: {output_path}")
    return output_path


output_pdf_path = save_ddr_as_pdf(ddr_text, "DDR_Report.pdf")

✅ DDR PDF saved: DDR_Report.pdf


In [ ]:
from google.colab import files
files.download(output_pdf_path)
print("📥 Download started!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Download started!
